# Graph editing

In this notebook we describe some fundamental graph manipulations that occur when transforming computational graphs.
More specifically, we describe two kind of manipulations:

* tree traversal and **leaf replacement**;
* graph morphisms and **algebraic graph rewriting**.


For didactic purposes, in the following cells we define some structures and utilities to print out informative plots.

In [1]:
from collections import namedtuple
from enum import unique, Enum, IntEnum


# template appearance of GraphViz components

GVNodeAppearance = namedtuple('GVNodeAppearance', ['fontsize', 'fontcolor',                   # node label
                                                   'penwidth', 'pencolor',                    # node boundary
                                                   'shape', 'height', 'width', 'fillcolor'])  # node body
GVEdgeAppearance = namedtuple('GVEdgeAppearance', ['penwidth', 'color'])


# node types and tree traversal

@unique
class NodeType(IntEnum):
    CONTAINER     = 0
    LINEAR        = 1
    NONLINEAR     = 2
    POOLING       = 3
    NORMALISATION = 4


style_node_type = {
    NodeType.CONTAINER:     GVNodeAppearance(fontsize='6', fontcolor='black',
                                             penwidth='1', pencolor='black',
                                             shape='circle', height='1.0', width='1.0', fillcolor='lightslategray'),
    NodeType.LINEAR:        GVNodeAppearance(fontsize='6', fontcolor='black',
                                             penwidth='1', pencolor='black',
                                             shape='circle', height='1.0', width='1.0', fillcolor='lightskyblue'),
    NodeType.NONLINEAR:     GVNodeAppearance(fontsize='6', fontcolor='black',
                                             penwidth='1', pencolor='black',
                                             shape='circle', height='1.0', width='1.0', fillcolor='lightseagreen'),
    NodeType.POOLING:       GVNodeAppearance(fontsize='6', fontcolor='black',
                                             penwidth='1', pencolor='black',
                                             shape='circle', height='1.0', width='1.0', fillcolor='lightsalmon'),
    NodeType.NORMALISATION: GVNodeAppearance(fontsize='6', fontcolor='black',
                                             penwidth='1', pencolor='black',
                                             shape='circle', height='1.0', width='1.0', fillcolor='lightgoldenrod')
}


@unique
class NodeContainerLeaf(IntEnum):
    CONTAINER = 0
    LEAF      = 1


style_node_containerleaf = {
    NodeContainerLeaf.CONTAINER: GVNodeAppearance(fontsize='6', fontcolor='black',
                                                  penwidth='1', pencolor='black',
                                                  shape='circle', height='1.0', width='1.0', fillcolor='sienna1'),
    NodeContainerLeaf.LEAF:      GVNodeAppearance(fontsize='6', fontcolor='darkgreen',
                                                  penwidth='2', pencolor='darkgreen',
                                                  shape='circle', height='1.0', width='1.0', fillcolor='palegreen3')
}


type_2_cl = {
    NodeType.CONTAINER:     NodeContainerLeaf.CONTAINER,
    NodeType.LINEAR:        NodeContainerLeaf.LEAF,
    NodeType.NONLINEAR:     NodeContainerLeaf.LEAF,
    NodeType.POOLING:       NodeContainerLeaf.LEAF,
    NodeType.NORMALISATION: NodeContainerLeaf.LEAF
}


# graph components highlights

@unique
class NodeState(IntEnum):
    INACTIVE  = 0
    ACTIVEPOS = 1
    ACTIVENEG = 2


style_node_state = {
    NodeState.INACTIVE:  GVNodeAppearance(fontsize='6', fontcolor='black',
                                          penwidth='1', pencolor='darkgray',
                                          shape='circle', height='1.0', width='1.0', fillcolor='gray'),
    NodeState.ACTIVEPOS: GVNodeAppearance(fontsize='6', fontcolor='darkgreen',
                                          penwidth='2', pencolor='darkgreen',
                                          shape='circle', height='1.0', width='1.0', fillcolor='chartreuse'),
    NodeState.ACTIVENEG: GVNodeAppearance(fontsize='6', fontcolor='crimson',
                                          penwidth='2', pencolor='crimson',
                                          shape='circle', height='1.0', width='1.0', fillcolor='firebrick2')
}


@unique
class EdgeState(IntEnum):
    INACTIVE  = 0
    ACTIVEPOS = 1
    ACTIVENEG = 2


style_edge_state = {
    EdgeState.INACTIVE:  GVEdgeAppearance(penwidth='1', color='gray'),
    EdgeState.ACTIVEPOS: GVEdgeAppearance(penwidth='2', color='darkgreen'),
    EdgeState.ACTIVENEG: GVEdgeAppearance(penwidth='2', color='crimson')
}


# graph rewriting rules

@unique
class NodeGRR(IntEnum):
    CONTEXT     = 0  # K-term
    TEMPLATE    = 1  # L-term
    REPLACEMENT = 2  # R-term


style_node_grr = {
    NodeGRR.CONTEXT:     GVNodeAppearance(fontsize='6', fontcolor='black',
                                          penwidth='1', pencolor='black',
                                          shape='circle', height='1.0', width='1.0', fillcolor='goldenrod'),
    NodeGRR.TEMPLATE:    GVNodeAppearance(fontsize='6', fontcolor='black',
                                          penwidth='1', pencolor='black',
                                          shape='circle', height='1.0', width='1.0', fillcolor='turquoise4'),
    NodeGRR.REPLACEMENT: GVNodeAppearance(fontsize='6', fontcolor='black',
                                          penwidth='1', pencolor='black',
                                          shape='circle', height='1.0', width='1.0', fillcolor='orange')
}


@unique
class EdgeGRR(IntEnum):
    CONTEXT             = 0  # K-term
    TEMPLATE            = 1  # L-term (to be removed)
    REPLACEMENT         = 2  # R-term (to be added)
    CONTEXT2TEMPLATE    = 3  # L-term (to be removed)
    CONTEXT2REPLACEMENT = 4  # R-term (to be added)


style_edge_grr = {
    EdgeGRR.CONTEXT:             GVEdgeAppearance(penwidth='2', color='goldenrod'),
    EdgeGRR.TEMPLATE:            GVEdgeAppearance(penwidth='2', color='turquoise4'),
    EdgeGRR.REPLACEMENT:         GVEdgeAppearance(penwidth='2', color='orange'),
    EdgeGRR.CONTEXT2TEMPLATE:    GVEdgeAppearance(penwidth='2', color='turquoise'),
    EdgeGRR.CONTEXT2REPLACEMENT: GVEdgeAppearance(penwidth='2', color='orangered2')
}


In [2]:
import os
import networkx as nx
import graphviz as gv
from IPython.display import display, IFrame

from typing import TypeVar
from typing import Union, Set, Tuple, List, Dict


NodeName = TypeVar('NodeName', int, str)


def nx_2_gv(G:            nx.DiGraph,
            revert_arcs:  bool = False,
            node_2_label: Dict[NodeName, str] = dict(),
            node_2_style: Dict[NodeName, GVNodeAppearance] = dict(),
            arc_2_style:  Dict[NodeName, GVEdgeAppearance] = dict()) -> gv.Digraph:
    """Convert a NetworkX graph into a PyGraphViz graph.
    
    GraphViz's annotation format allows descriptive renderings of graphs, which
    make it an ideal tool for didactic purposes. This function takes a NetworkX
    directed graph and produces a PyGraphViz equivalent representation,
    annotating the nodes and arcs as specified by labelling and style arguments.
    """
    
    gvG = gv.Digraph()
    
    for node in G.nodes:

        # retrieve node label
        try:
            label = node_2_label[node]
        except KeyError:
            label = ''

        # retrieve node style
        try:
            style = node_2_style[node]._asdict()
        except KeyError:
            style = {}

        gvG.node(str(node), label, **style, style='filled')
    
    for arc in G.edges:
        
        # retrieve edge style
        try:
            style = arc_2_style[arc]._asdict()
        except KeyError:
            style = {}

        s = arc[1] if revert_arcs else arc[0]
        e = arc[0] if revert_arcs else arc[1]
        gvG.edge(str(s), str(e), **style)  # print trees top-down in case leaves point to root, and not viceversa
    
    return gvG


def print_and_render(G:            nx.DiGraph,
                     filename:     str,
                     revert_arcs:  bool = False,
                     node_2_label: dict = dict(),
                     node_2_style: dict = dict(),
                     arc_2_style:  dict = dict(),
                     width:        int = 950,
                     height:       int = 300) -> None:
    
    dir_figures = os.path.join(os.path.curdir, 'figures')
    if not os.path.isdir(dir_figures):
        os.makedirs(dir_figures, exist_ok=True)

    gvG = nx_2_gv(G, revert_arcs=revert_arcs, node_2_label=node_2_label, node_2_style=node_2_style, arc_2_style=arc_2_style)
    gvG.render(directory=dir_figures, filename=filename)
    display(IFrame(os.path.join(dir_figures, filename + '.pdf'), width, height))


def show_highlighted_part(G:        nx.DiGraph,
                          filename: str,
                          VH:       Set[NodeName],
                          EH:       Union[Set[Tuple[NodeName, NodeName]], None] = None,
                          show_pos: bool = True,
                          width:    int = 950,
                          height:   int = 600) -> None:
    
    assert VH.issubset(set(G.nodes))
    
    if EH is not None:
        assert EH.issubset(set(G.edges))
    else:
        EH = set(G.subgraph(VH).edges)  # highlight the sub-graph induced by VH
        
    node_2_style = {n: style_node_state[NodeState.ACTIVEPOS if show_pos else NodeState.ACTIVENEG] if n in VH else style_node_state[NodeState.INACTIVE] for n in set(G.nodes)}
    arc_2_style  = {a: style_edge_state[EdgeState.ACTIVEPOS if show_pos else EdgeState.ACTIVENEG] if a in EH else style_edge_state[EdgeState.INACTIVE] for a in set(G.edges)}
    
    print_and_render(G, filename, node_2_style=node_2_style, arc_2_style=arc_2_style, width=width, height=height)


## Tree traversal and leaf replacement

To highlight the importance of tree traversal and leaf replacement in the context of QuantLab, we define a collection of very simplified data structures that make it possible to mimic the PyTorch network definition process.


In [3]:
from __future__ import annotations

from typing import TypeVar, NewType, Union, List, Tuple
from collections.abc import Iterator


ModuleType = TypeVar('ModuleType', bound=NodeType)
ModuleName = NewType('ModuleName', str)


class Module(object):
    
    def __init__(self, children: List[Module], type_: ModuleType, name: Union[ModuleName, None] = None) -> None:
        self._type          = type_
        self._name          = name
        self.named_children = dict()
        self._register_children(children)
    
    @property
    def type_(self) -> ModuleType:
        return self._type
    
    @property
    def name(self) -> Union[ModuleName, None]:
        return self._name

    def _register_children(self, children: List[Module]):

        children_names = [m.name for m in children if m.name is not None]
        assert len(children_names) in {0, len(children)}  # either all modules in the container have a name or none of them has

        if len(children_names) > 0:
            named_children = {n: m for n, m in zip(children_names, children)}
        else:
            named_children = {str(i): m for i, m in enumerate(children)}
            
        self.named_children = named_children
        
    @property
    def children(self) -> List[Module]:
        return [m for m in self.named_children.values()]
    

class Container(Module):
    
    def __init__(self, children: List[Module], name: Union[ModuleName, None] = None) -> None:
        assert len(children) > 0  # at least one module is required when building a `Container`
        super(Container, self).__init__(children=children, type_=NodeType.CONTAINER, name=name)
        

class Basic(Module):
    
    def __init__(self, type_: ModuleType, name: Union[ModuleName, None] = None) -> None:
        super(Basic, self).__init__(children=list(), type_=type_, name=name)
        

In [4]:
pilot      = Container([Basic(NodeType.LINEAR),
                        Basic(NodeType.NORMALISATION),
                        Basic(NodeType.NONLINEAR),
                        Basic(NodeType.POOLING)],
                       name=ModuleName('pilot'))
features   = Container([Basic(NodeType.LINEAR),
                        Basic(NodeType.NORMALISATION),
                        Basic(NodeType.NONLINEAR), Basic(NodeType.POOLING),
                        Basic(NodeType.LINEAR),
                        Basic(NodeType.NORMALISATION),
                        Basic(NodeType.NONLINEAR)],
                       name=ModuleName('features'))
pooling    = Basic(NodeType.POOLING, name=ModuleName('pooling'))
classifier = Container([Basic(NodeType.LINEAR),
                        Basic(NodeType.NORMALISATION),
                        Basic(NodeType.NONLINEAR),
                        Basic(NodeType.LINEAR)],
                       name=ModuleName('classifier'))
net        = Container([pilot,
                        features,
                        pooling,
                        classifier],
                       name=ModuleName('net'))


In [5]:
ModulePath = NewType('ModulePath', str)  # string of dot-separated `ModuleName`s


def get_network_tree(net: Module) -> nx.DiGraph:

    def traverse_tree(module: Module, module_name: ModulePath = ModulePath(''), G: nx.DiGraph() = nx.DiGraph()):

        if module_name == '':  # bootstrap tree traversal
            G.add_node(module_name, type_=module.type_, module=module)
        
        for child_name, child_module in module.named_children.items():
            full_name = '.'.join([module_name, child_name]) if module_name != '' else child_name
            G.add_node(full_name, type_=child_module.type_, module=child_module)
            G.add_edge(full_name, module_name)
            traverse_tree(child_module, module_name=full_name, G=G)
            
        return G
    
    return traverse_tree(net)


In [6]:
T = get_network_tree(net)

# show dichotomy container-leaf
print_and_render(T, 'T_containerleaf', revert_arcs=True, node_2_label={n: n for n in set(T.nodes)}, node_2_style={n: style_node_containerleaf[type_2_cl[type_]] for n, type_ in nx.get_node_attributes(T, 'type_').items()})

# show node types
print_and_render(T, 'T_nodetype', revert_arcs=True, node_2_label={n: n for n in set(T.nodes)}, node_2_style={n: style_node_type[type_] for n, type_ in nx.get_node_attributes(T, 'type_').items()})


In [7]:
from typing import NamedTuple


class LeafNode(NamedTuple):
    path:   ModulePath
    module: Module
        
    def __repr__(self) -> str:
        return "Path: {:20s} - Type: {}".format(self.path, self.module.type_)

    
def get_nodes_list(T: nx.DiGraph) -> List[LeafNode]:
    return [LeafNode(path=n, module=m) for n, m in nx.get_node_attributes(T, 'module').items() if len(list(T.predecessors(n))) == 0]


In [8]:
for ln in get_nodes_list(T):
    print(ln)
    

Path: pilot.0              - Type: 1
Path: pilot.1              - Type: 4
Path: pilot.2              - Type: 2
Path: pilot.3              - Type: 3
Path: features.0           - Type: 1
Path: features.1           - Type: 4
Path: features.2           - Type: 2
Path: features.3           - Type: 3
Path: features.4           - Type: 1
Path: features.5           - Type: 4
Path: features.6           - Type: 2
Path: pooling              - Type: 3
Path: classifier.0         - Type: 1
Path: classifier.1         - Type: 4
Path: classifier.2         - Type: 2
Path: classifier.3         - Type: 1


In [9]:
def replace_node(module: Module, path: ModulePath, newmodule: Module):
    
    try:
        child_name, subpath = path.split('.', 1)
        replace_node(module=module.named_children[child_name], path=subpath, newmodule=newmodule)
    except ValueError:
        child_name = path
        module.named_children[child_name] = newmodule


for ln in filter(lambda ln: ln.module.type_ == NodeType.LINEAR, get_nodes_list(T)):
    newmodule = Container([Basic(NodeType.NONLINEAR),
                           Basic(NodeType.LINEAR)])
    replace_node(net, ln.path, newmodule)
    

In [10]:
newT = get_network_tree(net)
    
# show dichotomy container-leaf
print_and_render(newT, 'newT_containerleaf', revert_arcs=True, node_2_label={n: n for n in set(T.nodes)}, node_2_style={n: style_node_containerleaf[type_2_cl[type_]] for n, type_ in nx.get_node_attributes(newT, 'type_').items()})

# show node types
print_and_render(newT, 'newT_nodetype', revert_arcs=True, node_2_label={n: n for n in set(T.nodes)}, node_2_style={n: style_node_type[type_] for n, type_ in nx.get_node_attributes(newT, 'type_').items()})


In [11]:
for ln in get_nodes_list(newT):
    print(ln)


Path: pilot.0.0            - Type: 2
Path: pilot.0.1            - Type: 1
Path: pilot.1              - Type: 4
Path: pilot.2              - Type: 2
Path: pilot.3              - Type: 3
Path: features.0.0         - Type: 2
Path: features.0.1         - Type: 1
Path: features.1           - Type: 4
Path: features.2           - Type: 2
Path: features.3           - Type: 3
Path: features.4.0         - Type: 2
Path: features.4.1         - Type: 1
Path: features.5           - Type: 4
Path: features.6           - Type: 2
Path: pooling              - Type: 3
Path: classifier.0.0       - Type: 2
Path: classifier.0.1       - Type: 1
Path: classifier.1         - Type: 4
Path: classifier.2         - Type: 2
Path: classifier.3.0       - Type: 2
Path: classifier.3.1       - Type: 1


## Graph morphisms and algebraic graph rewriting

The task of *graph rewriting* is concerned with transforming graphs into other graphs.
Therefore, it is an important topic in graph theory.


In [12]:
from collections import OrderedDict


# define nodes and connectivity of an example graph
G_node_2_type = OrderedDict([( 0, NodeType.LINEAR),
                             ( 1, NodeType.NORMALISATION),
                             ( 2, NodeType.NONLINEAR),
                             ( 3, NodeType.POOLING),
                             ( 4, NodeType.LINEAR),        ( 9, NodeType.LINEAR),
                             ( 5, NodeType.NORMALISATION), (10, NodeType.NORMALISATION),
                             ( 6, NodeType.NONLINEAR),
                             ( 7, NodeType.LINEAR),
                             ( 8, NodeType.NORMALISATION),
                             (11, NodeType.LINEAR),
                             (12, NodeType.NONLINEAR),
                             (13, NodeType.POOLING),
                             (14, NodeType.LINEAR),        (19, NodeType.LINEAR),
                             (15, NodeType.NORMALISATION), (20, NodeType.POOLING),
                             (16, NodeType.NONLINEAR),
                             (17, NodeType.LINEAR),
                             (18, NodeType.NORMALISATION),
                             (21, NodeType.LINEAR),
                             (22, NodeType.NONLINEAR),
                             (23, NodeType.POOLING),
                             (24, NodeType.LINEAR)])
VG                        = set(G_node_2_type.keys())  # nodes set
EG                        = {( 0,  1),                 # arcs set
                             ( 1,  2),
                             ( 2,  3),
                             ( 3,  4), ( 3,  9),
                             ( 4,  5), ( 9, 10),
                             ( 5,  6),
                             ( 6,  7),
                             ( 7,  8),
                             ( 8, 11), (10, 11),
                             (11, 12),
                             (12, 13),
                             (13, 14), (13, 19),
                             (14, 15), (19, 20),
                             (15, 16),
                             (16, 17),
                             (17, 18),
                             (18, 21), (20, 21),
                             (21, 22),
                             (22, 23),
                             (23, 24)}

# build (directed) graph
G = nx.DiGraph()
G.add_nodes_from(VG)
G.add_edges_from(EG)

# label nodes with type information
nx.set_node_attributes(G, G_node_2_type, 'type_')

# show graph
print_and_render(G, 'G_nodetype', node_2_style={n: style_node_type[type_] for n, type_ in nx.get_node_attributes(G, 'type_').items()}, height=700)


In [13]:
# define template L nodes and connectivity
L_node_2_type = OrderedDict([( 0, NodeType.NONLINEAR),
                             ( 1, NodeType.POOLING),
                             ( 2, NodeType.LINEAR),        ( 7, NodeType.LINEAR),
                             ( 3, NodeType.NORMALISATION), ( 8, NodeType.NORMALISATION),
                             ( 4, NodeType.NONLINEAR),
                             ( 5, NodeType.LINEAR),
                             ( 6, NodeType.NORMALISATION),
                             ( 9, NodeType.LINEAR),
                             (10, NodeType.NONLINEAR),
                             (11, NodeType.POOLING)])
VL                        = set(L_node_2_type.keys())
EL                        = {( 0,  1),
                             ( 1,  2), ( 1,  7),
                             ( 2,  3), ( 7,  8),
                             ( 3,  4),
                             ( 4,  5),
                             ( 5,  6),
                             ( 6,  9), ( 8,  9),
                             ( 9, 10),
                             (10, 11)}

# build (directed) graph
L = nx.DiGraph()
L.add_nodes_from(VL)
L.add_edges_from(EL)
nx.set_node_attributes(L, L_node_2_type, 'type_')

# relabel nodes to reflect the role they have in the GRR
VK = {0, 11}
nx.relabel_nodes(L, {n: '/'.join([('K' if n in VK else 'L') + '-term', str(n)]) for n in VL}, copy=False)
VL = set(L.nodes)
EL = set(L.edges)

# define context K as sub-graph of L
VK = {n for n in L.nodes if n.startswith('K')}
K  = L.subgraph(VK)  # in this case we use the induced sub-graph (https://en.wikipedia.org/wiki/Induced_subgraph), which has no edges
EK = set(K.edges)

# isolate "core" template L\K and context-template connections
VLK     = VL.difference(VK)
LK      = L.subgraph(VLK)
ELK     = set(LK.edges)
EK2LK2K = EL.difference(EK | ELK)

# define "core" replacement R\K
RK_node_2_type = OrderedDict([(12, NodeType.POOLING),
                              (13, NodeType.LINEAR),    (16, NodeType.LINEAR),
                              (14, NodeType.NONLINEAR),
                              (15, NodeType.LINEAR),
                              (17, NodeType.LINEAR),
                              (18, NodeType.NONLINEAR)])
VRK                        = set(RK_node_2_type.keys())
ERK                        = {(12, 13), (12, 16),
                              (13, 14),
                              (14, 15),
                              (15, 17), (16, 17),
                              (17, 18)}

# build (directed) graph
RK = nx.DiGraph()
RK.add_nodes_from(VRK)
RK.add_edges_from(ERK)
nx.set_node_attributes(RK, RK_node_2_type, 'type_')

# relabel nodes to reflect the role they have in the GRR
nx.relabel_nodes(RK, {n: '/'.join(['R-term', str(n)]) for n in VRK}, copy=False)
VRK = set(RK.nodes)
ERK = set(RK.edges)

# glue R\K to the context graph
S = nx.compose(L, RK)
EK2RK2K = {('K-term/0', 'R-term/12'), ('R-term/18', 'K-term/11')}
S.add_edges_from(EK2RK2K)
FK2RK = {vK: {a for a in EK2RK2K if a[0] == vK} for vK in VK}
FRK2K = {vK: {a for a in EK2RK2K if a[1] == vK} for vK in VK}

# identify the (full) replacement R
R  = S.subgraph(VK | VRK)
VR = set(R.nodes)
ER = set(R.edges)


In [14]:
# prepare the styles of the GRR's components
node_grr = dict()
node_grr.update({n: NodeGRR.CONTEXT     for n in VK})
node_grr.update({n: NodeGRR.TEMPLATE    for n in VLK})
node_grr.update({n: NodeGRR.REPLACEMENT for n in VRK})

arc_grr = dict()
arc_grr.update({a: EdgeGRR.CONTEXT             for a in EK})
arc_grr.update({a: EdgeGRR.TEMPLATE            for a in ELK})
arc_grr.update({a: EdgeGRR.REPLACEMENT         for a in ERK})
arc_grr.update({a: EdgeGRR.CONTEXT2TEMPLATE    for a in EK2LK2K})
arc_grr.update({a: EdgeGRR.CONTEXT2REPLACEMENT for a in EK2RK2K})

nx.set_node_attributes(S, node_grr, 'node_grr')
nx.set_edge_attributes(S, arc_grr,  'arc_grr')

# show the GRR parts
print_and_render(S, 'GRR_S_rule_parts', node_2_style={n: style_node_grr[grr] for n, grr in nx.get_node_attributes(S, 'node_grr').items()}, arc_2_style={a: style_edge_grr[grr] for a, grr in nx.get_edge_attributes(S, 'arc_grr').items()}, height=700)


In [15]:
show_highlighted_part(S, 'GRR_L', VL)

In [16]:
show_highlighted_part(S, 'GRR_L_core', VLK)

In [17]:
show_highlighted_part(S, 'GRR_K2LK2K_connections', VH=set(), EH=EK2LK2K)

In [18]:
show_highlighted_part(S, 'GRR_R_core', VH=VRK)

In [19]:
from networkx.algorithms import isomorphism


def find_morphisms(G: nx.DiGraph,
                   L: nx.DiGraph) -> List[Dict[NodeName, NodeName]]:

    matcher      = isomorphism.DiGraphMatcher(G, L)
    isomorphisms = list(matcher.subgraph_isomorphisms_iter())
    
    return isomorphisms


isomorphisms = find_morphisms(G, L)


In [20]:
show_highlighted_part(G, 'H1_typed', VH=set(isomorphisms[0].keys()), show_pos=True)

In [21]:
for vH, vL in isomorphisms[1].items():
    print("{:2d} ({:1d}) --> {:9s} ({:1d})".format(vH, G.nodes[vH]['type_'], vL, L.nodes[vL]['type_']))

show_highlighted_part(G, 'H2_untyped', VH=set(isomorphisms[1].keys()), show_pos=False)

12 (2) --> K-term/0  (2)
13 (3) --> L-term/1  (3)
14 (1) --> L-term/2  (1)
15 (4) --> L-term/3  (4)
16 (2) --> L-term/4  (2)
17 (1) --> L-term/5  (1)
18 (4) --> L-term/6  (4)
19 (1) --> L-term/7  (1)
20 (3) --> L-term/8  (4)
21 (1) --> L-term/9  (1)
22 (2) --> L-term/10 (2)
23 (3) --> K-term/11 (3)


Note that there is a mismatch between the types of node with ID `20` in the source graph `G` (type `P`) and node with ID `L-term/8` in the template graph `L` (type `BN`).

In [22]:
def find_typed_morphisms(G: nx.DiGraph,
                         L: nx.DiGraph) -> List[Dict[NodeName, NodeName]]:

    isomorphisms = find_morphisms(G, L)
    
    def is_typed_morphism(G: nx.DiGraph,
                          L: nx.DiGraph,
                          g: Dict[NodeName, NodeName]) -> bool:
        
        is_ok = True

        for vG, vL in g.items():
            is_ok = is_ok & (G.nodes[vG]['type_'] == L.nodes[vL]['type_'])
        
        return is_ok
    
    typed_isomorphisms = list(filter(lambda g: is_typed_morphism(G, L, g), isomorphisms))
    
    return typed_isomorphisms


In [23]:
typed_isomorphisms = find_typed_morphisms(G, L)
g = typed_isomorphisms[0]


Now, we proceed with the implementation of the rewriting rule.


In [24]:
# identify the interface I and the match core H\I

VI = {vG for vG, vL in g.items() if vL in VK}
I  = G.subgraph(VI)
EI = set(I.edges)

VHI = set(g.keys()).difference(VI)
HI  = G.subgraph(VHI)
EHI = set(HI.edges)

# generate substitute core J\I

JI = nx.relabel_nodes(RK, {vRK: '_'.join(['GRR', vRK.replace('R-term/', '')]) for vRK in VRK}, copy=True)
G  = nx.compose(G, JI)

# glue substitute core J\I to the interface I

JI2RK_morphisms = find_typed_morphisms(JI, RK)
assert len(JI2RK_morphisms) == 1
g_JI2RK = JI2RK_morphisms[0]
g_RK2JI = {vRK: vJI for vJI, vRK in g_JI2RK.items()}

for vI in VI:
    vK = g[vI]
    EvI2JI = {(vI, g_RK2JI[vRK]) for (_, vRK) in FK2RK[vK]}
    EJI2vI = {(g_RK2JI[vRK], vI) for (vRK, _) in FRK2K[vK]}
    G.add_edges_from(EvI2JI | EJI2vI)

# remove match core (NetworkX removes dangling edges automatically)

G.remove_nodes_from(HI)


In [25]:
print_and_render(G, 'G_nodetype_derived', node_2_style={n: style_node_type[type_] for n, type_ in nx.get_node_attributes(G, 'type_').items()}, height=700)